## 1. Import all libraries

In [4]:
import os
import json
import requests
import warnings
import sqlite3
from datetime import datetime, timedelta
from openai import OpenAI
from dotenv import load_dotenv

from urllib3.exceptions import NotOpenSSLWarning
warnings.simplefilter("ignore", NotOpenSSLWarning)

load_dotenv()

OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
SPOONACULAR_API_KEY = os.environ.get("SPOON_API_KEY")

## 2. Create cookies and headers
*These variables are used for getting the XHR file from the website*

In [5]:
cookies = {
    'EDEKA_PRIVACY': '1%40087%7C6%7C5030%40%4091%401759168772089%2C1759168772089%2C1792864772089%40',
    'EDEKA_PRIVACY_CENTER': '',
    'JSESSIONID': '868E369E9FD720B0A453D661690D2FE1',
    'atuserid': '%7B%22name%22%3A%22atuserid%22%2C%22val%22%3A%22OPT-OUT%22%2C%22options%22%3A%7B%22end%22%3A%222026-10-31T18%3A02%3A53.457Z%22%2C%22path%22%3A%22%2F%22%7D%7D',
    'TCPID': '12591202533522197673',
}

headers = {
    'accept': '*/*',
    'accept-language': 'en-US,en;q=0.9,de;q=0.8,vi;q=0.7',
    'if-none-match': '"cvpwz8bzw34oqn"',
    'priority': 'u=1, i',
    'referer': 'https://www.edeka.de/eh/s%C3%BCdwest/edeka-frank-erlachstra%C3%9Fe-45/angebote.jsp',
    'sec-ch-ua': '"Chromium";v="140", "Not=A?Brand";v="24", "Google Chrome";v="140"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-origin',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/140.0.0.0 Safari/537.36',
    # 'cookie': 'EDEKA_PRIVACY=1%40087%7C6%7C5030%40%4091%401759168772089%2C1759168772089%2C1792864772089%40; EDEKA_PRIVACY_CENTER=; JSESSIONID=868E369E9FD720B0A453D661690D2FE1; atuserid=%7B%22name%22%3A%22atuserid%22%2C%22val%22%3A%22OPT-OUT%22%2C%22options%22%3A%7B%22end%22%3A%222026-10-31T18%3A02%3A53.457Z%22%2C%22path%22%3A%22%2F%22%7D%7D; TCPID=12591202533522197673',
}

params = {
    'path': 'api/offers?limit=999&marketId=10001604',
}

In [6]:
def create_list_from_json(data):
	important_fields = ["title", "price", "category", "validTill"]
	allowed_category = ['Fleisch & Wurst', 'Fisch & Meeresfrüchte', 'Molkerei & Käse', 'Tiefkühl', 'Obst & Gemüse', 'Grundnahrung']
	reduced_items = []
	offers = data["offers"]
	for offer in offers:
		if ("Vegan" in offer['title']):
			continue
		if (offer["category"]["name"] not in allowed_category):
			continue	
		item = {}
		for field in important_fields:
			if field == "price":
				item[field] = offer[field]["value"]
			elif field == "category":
				item[field] = offer[field]["name"]
			elif field == "title":
				item["raw_title"] = offer[field]
			else:
				item[field] = offer[field]
		reduced_items.append(item)
	return reduced_items

## 4. Get the data and convert to JSON

In [7]:
response = requests.get('https://www.edeka.de/api/auth-proxy/', params=params, cookies=cookies, headers=headers)
data = response.json()
reduced_items = create_list_from_json(data)

In [8]:
def normalize_values(item_list):
	dt_format = "%Y-%m-%d"
	for item in item_list:
		item["raw_title"] = item["raw_title"].replace("\xa0", " ").strip()
		item["price"] = float(item["price"])
		item["validTill"] = datetime.strptime(item["validTill"], dt_format)
		if ("Ab Donnerstag" in item["raw_title"]):
			item["validFrom"] = item["validTill"] - timedelta(days=2)
			item["raw_title"] = item["raw_title"].replace("Ab Donnerstag erhältlich: ", "")
		elif ("Am Samstag" in item["raw_title"]):
			item["validFrom"] = item["validTill"]
			item["raw_title"] = item["raw_title"].replace("Am Samstag erhältlich: ", "")
		else:
			item["validFrom"] = item["validTill"] - timedelta(days=6)

normalize_values(reduced_items)

In [59]:
for item in reduced_items:
	print(item)

{'raw_title': 'Original Wagner Steinofen Pizza', 'price': 1.79, 'category': 'Tiefkühl', 'validTill': datetime.datetime(2025, 10, 4, 0, 0), 'validFrom': datetime.datetime(2025, 9, 28, 0, 0)}
{'raw_title': 'Meggle Butter', 'price': 1.99, 'category': 'Molkerei & Käse', 'validTill': datetime.datetime(2025, 10, 4, 0, 0), 'validFrom': datetime.datetime(2025, 9, 28, 0, 0)}
{'raw_title': 'Himbeeren', 'price': 1.99, 'category': 'Obst & Gemüse', 'validTill': datetime.datetime(2025, 10, 4, 0, 0), 'validFrom': datetime.datetime(2025, 9, 28, 0, 0)}
{'raw_title': 'EDEKA Genussmomente Mandarinen', 'price': 1.49, 'category': 'Obst & Gemüse', 'validTill': datetime.datetime(2025, 10, 4, 0, 0), 'validFrom': datetime.datetime(2025, 9, 28, 0, 0)}
{'raw_title': 'Frische Schweinelenden/ -filets', 'price': 1.11, 'category': 'Fleisch & Wurst', 'validTill': datetime.datetime(2025, 10, 4, 0, 0), 'validFrom': datetime.datetime(2025, 9, 28, 0, 0)}
{'raw_title': 'Frische Puten-Schnitzel oder -Brust', 'price': 1.39,

## 5. Use LLM from OpenAI to generate a JSON with clean item title
*This should be optimized to reduced the LLM runtime*

In [33]:
OpenAI.api_ley = OPENAI_API_KEY
client = OpenAI()

In [59]:
prompt = f"""
	Go through every dictionary item in the list ```{reduced_items}```\
	and extract the core ingredient or food item; this name should be translated to English.\
	For example:\
	- 'Original Wagner Steinofen Pizza' shoule be 'pizza'\
	- 'Weihenstephan haltbare Milch' should be 'milk'\
	Respond with JSON: create a list of dictionaries (key is products),\
	each has one key 'clean_name' with the value is the extracted ingredient.\
	Make sure that every item has one corresponding extracted name.\
	If you do not know the translation, leave the name emtpy ''.\
	There should be in total ```{len(reduced_items)}``` items.
"""

In [48]:
we_did_not_specify_stop_tokens = True

try:
	response = client.responses.create(
		model="gpt-4o-mini",
		input=[
			{"role": "system", "content": "You are a helpful assistant designed to output JSON."},
			{"role": "user", "content": prompt}],
		temperature=0.5,
		text={"format": {"type": "json_object"} }
	)
	# Check if the conversation was too long for the context window, resulting in incomplete JSON 
	if response.status == "incomplete" and response.incomplete_details.reason == "max_output_tokens":
		# your code should handle this error case
		pass

	# Check if the OpenAI safety system refused the request and generated a refusal instead
	if response.output[0].content[0].type == "refusal":
		# your code should handle this error case
		# In this case, the .content field will contain the explanation (if any) that the model generated for why it is refusing
		print(response.output[0].content[0]["refusal"])

	# Check if the model's output included restricted content, so the generation of JSON was halted and may be partial
	if response.status == "incomplete" and response.incomplete_details.reason == "content_filter":
		# your code should handle this error case
		pass

	# if response.status == "completed":
	# 	# In this case the model has either successfully finished generating the JSON object according to your schema, or the model generated one of the tokens you provided as a "stop token"

	# 	if we_did_not_specify_stop_tokens:
	# 		# If you didn't specify any stop tokens, then the generation is complete and the content key will contain the serialized JSON object
	# 		# This will parse successfully and should now contain  "{"winner": "Los Angeles Dodgers"}"
	# 		print(response.output_text)
	# 	else:
	# 		# Check if the response.output_text ends with one of your stop tokens and handle appropriately
	# 		pass
except Exception as e:
    # Your code should handle errors here, for example a network error calling the API
    print(e)

In [ ]:
## can use langchain for creating the json format and calling LLM
 
from langchain.chat_models import init_chat_model
from langchain.output_parsers.json import SimpleJsonOutputParser
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import OpenAI
from pydantic import BaseModel, Field, model_validator


model = OpenAI(model_name="gpt-4o-mini", temperature=0.0)

class DiscountItem(BaseModel):
	raw_title: str = Field(description="Item title based on the supermarket description")
	price: str = Field(description="Reduced price")
	category: str = Field(description="Item category based on the supermarket description")
	valid_till: str = Field(description="price")
	validFrom: str = Field(description="Item title based on the supermarket description")
	clean_title: str = Field(description="Item title used in LLM")

    # You can add custom validation logic easily with Pydantic.
    # @model_validator(mode="before")
    # @classmethod
    # def question_ends_with_question_mark(cls, values: dict) -> dict:
    #     setup = values.get("setup")
    #     if setup and setup[-1] != "?":
    #         raise ValueError("Badly formed question!")
    #     return values
    

json_prompt = PromptTemplate.from_template(
    "Return a JSON object with an `answer` key that answers the following question: {question}"
)
json_parser = SimpleJsonOutputParser()
json_chain = json_prompt | model | json_parser


In [49]:
json_clean_name = json.loads(response.output_text)
try:
	clean_name_list = json_clean_name["products"]
	for i in range(len(reduced_items)):
		reduced_items[i].update(clean_name_list[i])
except Exception as e:
	print(e)

## 6. Connect a recipe API (Spoonacular) to query some recipes

In [ ]:
# Creating table using SQL

con = sqlite3.connect("items.db")
cur = con.cursor()
cur.execute("CREATE TABLE products(id INTEGER PRIMARY KEY AUTOINCREMENT, raw_title TEXT, category TEXT, price REAL, valid_from DATE, valid_till TEXT, clean_name TEXT, source TEXT)")
con.commit()
for item in reduced_items:
	cur.execute("""
	INSERT INTO products (raw_title, clean_name, category, price, valid_from, valid_till, source)
	VALUES (?, ?, ?, ?, ?, ?, ?)
    """, (
        item["raw_title"], item["clean_name"], item["category"], 
		item["price"], item["validFrom"], item["validTill"], "EDEKA"
    ))
con.commit()

OperationalError: table products already exists

In [65]:
con = sqlite3.connect("items.db")
cur = con.cursor()

cur.execute("SELECT clean_name FROM products WHERE CATEGORY='Fleisch & Wurst'")
meat = [row[0] for row in cur.fetchall()]
cur.execute("SELECT clean_name FROM products WHERE CATEGORY='Fisch & Meeresfrüchte'")
fish = [row[0] for row in cur.fetchall()]
cur.execute("SELECT clean_name FROM products WHERE CATEGORY='Molkerei & Käse'")
cheese_n_milk = [row[0] for row in cur.fetchall()]
cur.execute("SELECT clean_name FROM products WHERE CATEGORY='Obst & Gemüse'")
veggi = [row[0] for row in cur.fetchall()]
cur.execute("SELECT clean_name FROM products WHERE CATEGORY='Grundnahrung'")
bread = [row[0] for row in cur.fetchall()]
cur.execute("SELECT clean_name FROM products WHERE CATEGORY='Tiefkühl'")
frozen = [row[0] for row in cur.fetchall()]
con.close()

In [44]:
params = {
    "ingredients": ",".join(ingredients[:10]),  # limit to a handful
    "number": 10,
    "apiKey": SPOONACULAR_API_KEY,
}
headers = {
    'Content-Type': 'application/json',
}
resp = requests.get("https://api.spoonacular.com/recipes/findByIngredients", params=params, headers=headers)
recipes = resp.json()

In [46]:
y =json.dumps(recipes)
print(y)

[{"id": 658134, "title": "Redneck Breakfast on a Croissant", "image": "https://img.spoonacular.com/recipes/658134-312x231.jpg", "imageType": "jpg", "usedIngredientCount": 2, "missedIngredientCount": 2, "missedIngredients": [{"id": 18239, "amount": 1.0, "unit": "", "unitLong": "", "unitShort": "", "aisle": "Bakery/Bread", "name": "croissant", "original": "1 croissant", "originalName": "croissant", "meta": [], "image": "https://img.spoonacular.com/ingredients_100x100/croissants.jpg"}, {"id": 1123, "amount": 1.0, "unit": "", "unitLong": "", "unitShort": "", "aisle": "Milk, Eggs, Other Dairy", "name": "egg", "original": "1 egg", "originalName": "egg", "meta": [], "image": "https://img.spoonacular.com/ingredients_100x100/egg.png"}], "usedIngredients": [{"id": 13346, "amount": 0.25, "unit": "cup", "unitLong": "cups", "unitShort": "cup", "aisle": "Meat", "name": "corned beef hash", "original": "1/4 cup canned corned beef hash", "originalName": "canned corned beef hash", "meta": ["canned"], "e